In [2]:
%cd ..

/home/tae898/repos/erc


In [3]:
from glob import glob
import os
import json
import numpy as np
from pprint import pprint
import yaml
from src import read_json, read_yaml, parse_path, print_what_you_want

In [4]:
DATASET = 'MELD'
seed_results = glob(f'results/{DATASET}/*/*/kwargs.yaml')
seed_results = [parse_path(foo) for foo in seed_results]
seed_results = [foo for foo in seed_results if foo is not None]

print_what_you_want(seed_results, metric='f1_weighted')

ADD_BOU: True
ADD_EOU: True
ADD_SPEAKER_TOKENS: True
NUM_TRAIN_EPOCHS: 5
REPLACE_NAMES_IN_UTTERANCES: False
SPEAKER_SPLITS: ['train', 'val', 'test']
model_checkpoint: roberta-large
num_future_utterances: 1000
num_past_utterances: 1000
path: results/MELD/roberta-large/2021-06-02-11-56-09/kwargs.yaml
val_results_mean_std: {'mean': 0.6724572546276034, 'std': 0.0031377299187237894, 'num_samples': 5}
test_results_mean_std: {'mean': 0.6596394257273535, 'std': 0.006374465023475231, 'num_samples': 5}

ADD_BOU: False
ADD_EOU: False
ADD_SPEAKER_TOKENS: True
NUM_TRAIN_EPOCHS: 5
REPLACE_NAMES_IN_UTTERANCES: False
SPEAKER_SPLITS: ['train', 'val', 'test']
model_checkpoint: roberta-large
num_future_utterances: 1000
num_past_utterances: 1000
path: results/MELD/roberta-large/2021-06-01-12-36-45/kwargs.yaml
val_results_mean_std: {'mean': 0.6724088682011764, 'std': 0.009060874948742302, 'num_samples': 5}
test_results_mean_std: {'mean': 0.6578266286303185, 'std': 0.006683818890420509, 'num_samples': 5}

A

In [4]:
DATASET = 'IEMOCAP'
seed_results = glob(f'results/{DATASET}/*/*/kwargs.yaml')
seed_results = [parse_path(foo) for foo in seed_results]
seed_results = [foo for foo in seed_results if foo is not None]

print_what_you_want(seed_results, metric='f1_weighted')

ADD_BOU: False
ADD_EOU: False
ADD_SPEAKER_TOKENS: True
NUM_TRAIN_EPOCHS: 5
REPLACE_NAMES_IN_UTTERANCES: False
SPEAKER_SPLITS: ['train', 'val', 'test']
model_checkpoint: roberta-large
num_future_utterances: 0
num_past_utterances: 1000
path: results/IEMOCAP/roberta-large/2021-05-31-20-58-35/kwargs.yaml
val_results_mean_std: {'mean': 0.6434923303451727, 'std': 0.013105741362095797, 'num_samples': 5}
test_results_mean_std: {'mean': 0.6707066471947625, 'std': 0.008771320774317344, 'num_samples': 5}

ADD_BOU: False
ADD_EOU: False
ADD_SPEAKER_TOKENS: True
NUM_TRAIN_EPOCHS: 5
REPLACE_NAMES_IN_UTTERANCES: False
SPEAKER_SPLITS: ['train', 'val', 'test']
model_checkpoint: roberta-large
num_future_utterances: 1000
num_past_utterances: 1000
path: results/IEMOCAP/roberta-large/2021-06-01-03-03-58/kwargs.yaml
val_results_mean_std: {'mean': 0.6366509934734441, 'std': 0.008542158586976677, 'num_samples': 5}
test_results_mean_std: {'mean': 0.6688823566871317, 'std': 0.006678758101777102, 'num_samples': 5